## Demo notebook for ECOD DomainMapper analysis
###### Last updated 2022-12-01
This notebook offers a simple example of an analysis accessible using ECOD DomainMapper domains. Note here we're simply demonstrating some design patterns and identifying the set of proteins with three or more RRMs (RNA recognition motifs) that lack any other identified ECOD domains.

In [1]:
from shephard import interfaces
from shephard.apis import uniprot

In [2]:
# set this to the local location of the directory associated
# with https://github.com/holehouse-lab/shephard-data/tree/main/data/proteomes/human
data_directory = 'MUST_BE_UPDATED'


# this defines the local locations 
proteome_file = f'{data_directory}/human_proteome_clean.fasta'
domain_file   = f'{data_directory}/shprd_domains_ecod_domainmapper.tsv'
IDR_file   = f'{data_directory}/shprd_domains_idrs.tsv'




In [3]:
# read human proteome into a SHEPHARD proteome and annotate with
# the ECOD-DomainMapper domains
human_proteome = uniprot.uniprot_fasta_to_proteome(proteome_file)
interfaces.si_domains.add_domains_from_file(human_proteome, domain_file)
interfaces.si_domains.add_domains_from_file(human_proteome, IDR_file)

In [4]:
# first some exploration - how many unique domain types do we have in the human proteom?
all_domain_types = {}
for protein in human_proteome:
    for domain in protein.domains:
        
        # filter for DomainMapper domains
        if domain.attribute('from_domain_mapper', safe=False) == 'True':
            
            name = domain.attribute('domain_type')
            if name not in all_domain_types:
                all_domain_types[name] = 0
            all_domain_types[name] = all_domain_types[name] + 1
            
print(f'There are {len(all_domain_types)} distinct domain types in the human proteome')

There are 4798 distinct domain types in the human proteome


In [5]:
# get the domain types that are RRM-associated 
rrm_domain_names = []
for name in all_domain_types:
    if name.lower().find('rrm') > -1:
        rrm_domain_names.append(name)
print('List of RRM-associated names:')
print(rrm_domain_names)        

List of RRM-associated names:
['RRM_1_like', 'RRM_1_6', 'RRM_1_4', 'RRM_1_5', 'RRM_1_1', 'RRM_1', 'RRM_1_3', 'RRM_1_2', 'RRM_1_9', 'RRM_1_7', 'RRM_1_8', 'RRM_4', 'RRM_3', 'RRM_1_like_1', 'Nup35_RRM']


In [10]:
# find the UniProt accessions for proteins with one or more RRM
RRM_UIDs = set([])

for protein in human_proteome:
    for domain in protein.domains:
        
        # filter for DomainMapper domains
        if domain.attribute('from_domain_mapper', safe=False) == 'True':
            if domain.attribute('domain_type') in rrm_domain_names:
                RRM_UIDs.add(domain.protein.unique_ID)
                
print(f'There are {len(RRM_UIDs)} RRM-containing proteins in the human proteome')

There are 236 RRM-containing proteins in the human proteome


In [8]:
# for those proteins count the number of RRMs and the identity of the non-RRM domains 
# in a convenient format case we're interested in that ...
for uid in RRM_UIDs:
    
    protein = human_proteome.protein(uid)
    n_rrms = 0
    non_rrm_domains = []
    for domain in protein.domains:
        
        
        if domain.attribute('from_domain_mapper', safe=False) == 'True':
            if domain.attribute('domain_type') in rrm_domain_names:
                n_rrms = n_rrms + 1
            else:
                non_rrm_domains.append(f"{domain.attribute('domain_type')}-{domain.attribute('ECOD_T_group')}")
                
    # note we create some new protein-wide annotations here             
    protein.add_attribute('n_rrms',n_rrms, safe=False)
    protein.add_attribute('non_rrm_domains',non_rrm_domains, safe=False)
        
    
    
    

In [9]:
# finally, print names of proteins that posses 3 or more RRMs and no other annotated domains
# - these are inherently multivalent RNA binders and we would predict are likely associated 
# with biomolecular condensates if present at sufficient copy number
multivalent_RRM_proteins = []
for uid in RRM_UIDs:
    
    protein = human_proteome.protein(uid)
    if protein.attribute('n_rrms') > 2:
        if len(protein.attribute('non_rrm_domains')) == 0:
            multivalent_RRM_proteins.append(protein)
            print(protein.name)
            


sp|Q8IXT5|RB12B_HUMAN RNA-binding protein 12B OS=Homo sapiens OX=9606 GN=RBM12B PE=1 SV=2
sp|Q9Y4C8|RBM19_HUMAN Probable RNA-binding protein 19 OS=Homo sapiens OX=9606 GN=RBM19 PE=1 SV=3
sp|Q5SZQ8|CELF3_HUMAN CUGBP Elav-like family member 3 OS=Homo sapiens OX=9606 GN=CELF3 PE=1 SV=1
sp|P26599|PTBP1_HUMAN Polypyrimidine tract-binding protein 1 OS=Homo sapiens OX=9606 GN=PTBP1 PE=1 SV=1
sp|Q8TBY0|RBM46_HUMAN Probable RNA-binding protein 46 OS=Homo sapiens OX=9606 GN=RBM46 PE=1 SV=1
sp|Q01085|TIAR_HUMAN Nucleolysin TIAR OS=Homo sapiens OX=9606 GN=TIAL1 PE=1 SV=1
sp|O95319|CELF2_HUMAN CUGBP Elav-like family member 2 OS=Homo sapiens OX=9606 GN=CELF2 PE=1 SV=1
sp|Q9UKA9|PTBP2_HUMAN Polypyrimidine tract-binding protein 2 OS=Homo sapiens OX=9606 GN=PTBP2 PE=1 SV=1
sp|Q96DU9|PABP5_HUMAN Polyadenylate-binding protein 5 OS=Homo sapiens OX=9606 GN=PABPC5 PE=1 SV=1
sp|Q8IUH3|RBM45_HUMAN RNA-binding protein 45 OS=Homo sapiens OX=9606 GN=RBM45 PE=1 SV=1
sp|Q14576|ELAV3_HUMAN ELAV-like protein 3 OS=Ho